<a href="https://colab.research.google.com/github/Kaustuv47/Automatisation-of-Mobile-Health-Care-System/blob/master/System_For_FND_on_Dataset_Performance_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

Modules

In [ ]:
import pandas as pd
from google.colab import output
from collections import Counter
import numpy as np
# K-Mean
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import warnings
from sklearn.metrics import precision_recall_fscore_support
import pickle
import openpyxl
from openpyxl import load_workbook
from tensorflow.keras.callbacks import EarlyStopping

Dataset

In [ ]:
sheetUrl = "https://docs.google.com/spreadsheets/d/1azAfOVkkz42F3V9e0P9N5LHD4YbO5P--CFjIKBIxbEc/edit?usp=sharing"
sheetId = max(sheetUrl.split('/'), key=len)
sheetName = 'Train'
sheetUrl = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(sheetId, sheetName)
train_spreadsheet = pd.read_csv(sheetUrl)
sheetName = 'Test'
sheetUrl = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(sheetId, sheetName)
test_spreadsheet = pd.read_csv(sheetUrl)

spreadsheet = pd.concat([train_spreadsheet, test_spreadsheet], ignore_index=True)
spreadsheet = spreadsheet.drop_duplicates()
spreadsheet = spreadsheet.dropna()
spreadsheet = spreadsheet.reset_index(drop=True)
print(len(spreadsheet))

22724


In [ ]:
new_sheet = pd.DataFrame(columns=["id",	"title",	"author",	"text",	"label"])
with pd.ExcelWriter('NewDataSheet.xlsx', engine='openpyxl') as writer:
  new_sheet.to_excel(writer, sheet_name='Sheet1', index=False)
def writetofile(dataframe):
  file_path = 'NewDataSheet.xlsx'
  wb = load_workbook(file_path)
  sheet = wb.active
  for _, row in dataframe.iterrows():
    sheet.append(row.tolist())
  wb.save(file_path)
  wb.close()
  print("Done Writing")

In [ ]:
import openpyxl
workbook = openpyxl.Workbook()
worksheet = workbook.active
header = [i for i in range(1, 2)]
worksheet.append(['Partition'] + header)
workbook.save('Accuracy.xlsx')

def write(turn, row_list):
  workbook = openpyxl.load_workbook('Accuracy.xlsx')
  worksheet = workbook.active
  row_size = []
  row_accuracy = []
  for item in row_list:
    row_size.append(item["size"])
    row_accuracy.append(item["accuracy"])
  worksheet.append([turn] + row_size)
  worksheet.append(["Accuracy"] + row_accuracy)
  workbook.save('Accuracy.xlsx')

In [ ]:
class Models:
  def __init__(self, grouped_sheet):
    # global new_sheet
    store_sheet = grouped_sheet
    independent_feature = grouped_sheet.drop('label', axis=1)
    dependent_feature = grouped_sheet['label']
    voc_size=100000
    independent_feature.reset_index(inplace=True)
    nltk.download('stopwords')
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(independent_feature)):
        review = re.sub('[^a-zA-Z]', ' ', independent_feature['title'][i])
        review = review.lower()
        review = review.split()
        
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    independent_feature = [one_hot(words,voc_size)for words in corpus]
    independent_feature[1]
    sent_length = 100
    embedded_docs = pad_sequences(independent_feature,padding='pre',maxlen=sent_length)
    embedding_vector_features=40
    model=Sequential()
    model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
    model.add(Bidirectional(LSTM(100)))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    independent_feature_train = np.array(embedded_docs[:-1])
    dependent_feature_train = np.array(dependent_feature.iloc[:-1])
    independent_feature_test = np.array(embedded_docs[-1:])
    dependent_feature_test = np.array([dependent_feature.iloc[-1]])
    independent_feature_train.shape, dependent_feature_train.shape
    X_final=np.array(embedded_docs)
    y_final=np.array(dependent_feature)
    independent_feature_train, independent_feature_test, dependent_feature_train, dependent_feature_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
    model.fit(independent_feature_train, dependent_feature_train, validation_data = (independent_feature_test, dependent_feature_test), epochs=40, batch_size=64, callbacks=[early_stopping])
    predict_y = model.predict(independent_feature_test)
    y_pred1 = np.argmax(predict_y,axis=1)
    self.accuracy = accuracy_score(dependent_feature_test, y_pred1)
    if self.accuracy >= 0.80:
      store_sheet['id'] = 0
      print("Writing to file")
      writetofile(store_sheet)

  def result(self):
    return self.accuracy

In [ ]:
class FNDSystem:
  def __init__(self, sheet, clusters):
     self.sheet = sheet
     self.clusters = clusters
  
  def kmean_clusterer(self):
    paragraph_list = self.sheet['text'].tolist()
    vectorizer = TfidfVectorizer(stop_words='english')
    vectored_data = vectorizer.fit_transform(paragraph_list)
    kmeans = KMeans(n_clusters = self.clusters, random_state = 0).fit(vectored_data)
    count_dict = Counter(kmeans.labels_)
    # print(kmeans.labels_, count_dict)
    return self.grouper(kmeans.labels_)
  
  def grouper(self, labels):
    result_list = []
    for index, label in enumerate(labels):
      self.sheet.loc[index, 'id'] = label
    self.sheet = self.sheet.groupby('id')
    for index in range(self.clusters):
      size = self.sheet.get_group(list(self.sheet.groups.keys())[index]).shape[0]
      model_object = Models(self.sheet.get_group(list(self.sheet.groups.keys())[index]))
      result_list.append({"size": size, "accuracy": model_object.result()})
    return result_list

In [ ]:
for turn in range(1, 4):
  print("Round", turn)
  FND_object = FNDSystem(spreadsheet, turn)
  result_list = FND_object.kmean_clusterer()
  write(turn, result_list)
  print("Result", result_list)
  del FND_object
# writetofile()
print("Final Model")
file_path = 'NewDataSheet.xlsx'
newSheet = pd.read_excel(file_path)
final_model = Models(newSheet)
write("Final", [{"size": newSheet.shape[0], "accuracy": final_model.result()}])
print("Result", [{"size": newSheet.shape[0], "accuracy": final_model.result()}])

Round 1


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/40
238/238 [==============================] - 21s 69ms/step - loss: 0.4357 - accuracy: 0.7974 - val_loss: 0.3424 - val_accuracy: 0.8660
Epoch 2/40
238/238 [==============================] - 6s 24ms/step - loss: 0.2898 - accuracy: 0.8900 - val_loss: 0.3562 - val_accuracy: 0.8624
Epoch 3/40
238/238 [==============================] - 4s 17ms/step - loss: 0.2229 - accuracy: 0.9211 - val_loss: 0.3908 - val_accuracy: 0.8513
Epoch 4/40
238/238 [==============================] - 5s 22ms/step - loss: 0.1697 - accuracy: 0.9404 - val_loss: 0.4456 - val_accuracy: 0.8368
Epoch 5/40
238/238 [==============================] - 4s 16ms/step - loss: 0.1244 - accuracy: 0.9571 - val_loss: 0.5416 - val_accuracy: 0.8258
Epoch 6/40
238/238 [==============================] - 4s 16ms/step - loss: 0.0947 - accuracy: 0.9684 - val_loss: 0.5899 - val_accuracy: 0.8224
Epoch 6: early stopping
235/235 [==============================] - 2s 4ms/step
Result [{'size': 22724, 'accuracy': 0.5587411654887319}]
Round

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/40
214/214 [==============================] - 22s 84ms/step - loss: 0.4617 - accuracy: 0.7843 - val_loss: 0.3570 - val_accuracy: 0.8570
Epoch 2/40
214/214 [==============================] - 5s 22ms/step - loss: 0.3154 - accuracy: 0.8786 - val_loss: 0.3635 - val_accuracy: 0.8597
Epoch 3/40
214/214 [==============================] - 4s 19ms/step - loss: 0.2456 - accuracy: 0.9107 - val_loss: 0.4355 - val_accuracy: 0.8367
Epoch 4/40
214/214 [==============================] - 5s 22ms/step - loss: 0.1894 - accuracy: 0.9307 - val_loss: 0.4706 - val_accuracy: 0.8309
Epoch 5/40
214/214 [==============================] - 4s 18ms/step - loss: 0.1409 - accuracy: 0.9488 - val_loss: 0.5588 - val_accuracy: 0.8196
Epoch 6/40
214/214 [==============================] - 4s 17ms/step - loss: 0.1039 - accuracy: 0.9632 - val_loss: 0.6398 - val_accuracy: 0.8102
Epoch 7/40
214/214 [==============================] - 4s 19ms/step - loss: 0.0795 - accuracy: 0.9726 - val_loss: 0.6956 - val_accuracy: 0.799

...

Epoch 1/40
23/23 [==============================] - 8s 210ms/step - loss: 0.3239 - accuracy: 0.9357 - val_loss: 0.1599 - val_accuracy: 0.9670
Epoch 2/40
23/23 [==============================] - 2s 74ms/step - loss: 0.1383 - accuracy: 0.9710 - val_loss: 0.1469 - val_accuracy: 0.9670
Epoch 3/40
23/23 [==============================] - 1s 54ms/step - loss: 0.1338 - accuracy: 0.9710 - val_loss: 0.1442 - val_accuracy: 0.9670
Epoch 4/40
23/23 [==============================] - 2s 81ms/step - loss: 0.1318 - accuracy: 0.9710 - val_loss: 0.1442 - val_accuracy: 0.9670
Epoch 5/40
23/23 [==============================] - 1s 32ms/step - loss: 0.1303 - accuracy: 0.9710 - val_loss: 0.1426 - val_accuracy: 0.9670
Epoch 6/40
23/23 [==============================] - 1s 31ms/step - loss: 0.1295 - accuracy: 0.9710 - val_loss: 0.1420 - val_accuracy: 0.9670
Epoch 6: early stopping
22/22 [==============================] - 1s 6ms/step
Writing to file


<ipython-input-18-0a1d2e9fbefb>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_sheet['id'] = 0


Done Writing
Result [{'size': 17446, 'accuracy': 0.5606113233761723}, {'size': 3166, 'accuracy': 0.21626794258373205}, {'size': 2112, 'accuracy': 0.9670014347202296}]
Final Model


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/40
140/140 [==============================] - 15s 80ms/step - loss: 0.1776 - accuracy: 0.9601 - val_loss: 0.1389 - val_accuracy: 0.9637
Epoch 2/40
140/140 [==============================] - 4s 25ms/step - loss: 0.0675 - accuracy: 0.9794 - val_loss: 0.0311 - val_accuracy: 0.9902
Epoch 3/40
140/140 [==============================] - 4s 26ms/step - loss: 0.0145 - accuracy: 0.9966 - val_loss: 0.0178 - val_accuracy: 0.9957
Epoch 4/40
140/140 [==============================] - 3s 18ms/step - loss: 0.0048 - accuracy: 0.9990 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 5/40
140/140 [==============================] - 2s 18ms/step - loss: 0.0035 - accuracy: 0.9991 - val_loss: 0.0028 - val_accuracy: 0.9993
Epoch 6/40
140/140 [==============================] - 2s 16ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 7/40
140/140 [==============================] - 3s 20ms/step - loss: 7.7882e-04 - accuracy: 0.9998 - val_loss: 0.0011 - val_accuracy: 0

In [ ]:
print(classification_report(dependent_feature_test,y_pred1))

In [ ]:
  warnings.filterwarnings('always')
metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(dependent_feature_test, y_pred1, zero_division='warn')